In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time

import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import numpy as np

from IPython.core.display import display,HTML
from geopy.geocoders import ArcGIS

C:\Users\mdima\AppData\Local\Temp\ipykernel_11340\3638562896.py:13: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display,HTML


In [2]:
def click(x):
    button = driver.find_element(By.XPATH, x)
    button.click()
    time.sleep(2)

In [3]:
nom = ArcGIS()

get url link for all states

In [12]:
llink1 = []
url = "https://locations.traderjoes.com"
response = requests.get(url)
contents = response.content
soup = bs4(contents, "html.parser")
itemlist = soup.find_all("div",{"class":"itemlist"})

for i in itemlist:
    link = i.find("a").get("href")
    llink1.append(link)
    state = i.find("a").text
    lstate.append(state)

get url link for all cities in each state

In [13]:
llink2 = []
for i in llink1:
    r = requests.get(i)
    c = r.content
    s = bs4(c, "html.parser")
    item = s.find_all("div",{"class":"itemlist"})
    for j in item:
        link = j.find("a").get("href")
        llink2.append(link)
        city = j.find("a").text
        lcity.append(city)

get store info for all stores in each city

In [28]:
laddress, lcity, lstate, lzip, llocation = [],[],[],[],[]
for i in llink2:
    r = requests.get(i)
    c = r.content
    s = bs4(c, "html.parser")
    container = s.find("div",{"class":"breadcrumb container"})

    info = s.find_all("div",{"class":"address-left"})
    for j in info:
        state = container.find_all("a")[2].text
        lstate.append(state)
        city = container.find("span",{"aria-current":"page"}).text
        lcity.append(city)
        address = j.find_all("span")[1].text
        laddress.append(address)
        state_s = j.find_all("span")[3].text
        zipcode = j.find_all("span")[4].text
        lzip.append(zipcode)
        country = j.find_all("span")[5].text
        location = address+", "+city+", "+state_s+", "+zipcode+", "+country
        llocation.append(location)

create a dataframe

In [70]:
df = pd.DataFrame()
df["address"] = laddress
df["city"] = lcity
df["state"] = lstate
df["zipcode"] = lzip
df["location"] = llocation
df["coordinates"] = df["location"].apply(nom.geocode)
df["latitude"] = df["coordinates"].apply(lambda x: x.latitude if x != None else None)
df["longitude"] = df["coordinates"].apply(lambda x: x.longitude if x != None else None)

In [71]:
df = df.drop(columns = ["coordinates", "location"])

In [62]:
df.shape

(547, 7)

In [72]:
df.to_csv("trader_joes_store_data.csv", index = False)

Detailed explanation on how to make gspread work in your code with your google account, see [here](https://docs.gspread.org/en/latest/oauth2.html)

In [39]:
import gspread

In [43]:
gc = gspread.service_account(filename = "./../../service_account.json")
wks = gc.open("trader joe").sheet1

In [73]:
df.head(10)

,address,city,state,zipcode,latitude,longitude
0,"205 Summit Blvd, Suite 100",Birmingham,Alabama (AL),35243,33.447667,-86.728814
1,1100 Mid City Dr NW,Huntsville,Alabama (AL),35806,34.743765,-86.684235
2,11500 Financial Centre Pky,Little Rock,Arkansas (AR),72211,34.748223,-92.401625
3,1779 E. Williams Field Rd.,Gilbert,Arizona (AZ),85295,33.306444,-111.751715
4,7720 West Bell Rd,Glendale,Arizona (AZ),85308,33.639274,-112.226573
5,2050 E Baseline Rd,Mesa,Arizona (AZ),85204,33.380806,-111.787196
6,7912 N Oracle,Oro Valley,Arizona (AZ),85704,32.350491,-110.973229
7,4025 E Chandler Blvd,Phoenix,Arizona (AZ),85048,33.303962,-111.994431
8,4726 East Shea Blvd,Phoenix,Arizona (AZ),85028,33.583181,-111.979145
9,4821 N 20th St,Phoenix,Arizona (AZ),85016,33.507185,-112.037875


put dataframe values into google spreadsheet

In [1]:
for i in range(df.shape[1]):
    wks.update_cell(1,i+1, df.columns.tolist()[i])
    time.sleep(1)

for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        wks.update_cell(i+2,j+1, df.iloc[i,j])
        time.sleep(1)